In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input, BatchNormalization, Dense, Add, Activation, Reshape, Permute, Flatten
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.optimizers import Adam
from keras.applications import VGG19
import datetime
import random
from PIL import Image
import glob
import cv2
import matplotlib.pyplot as plt


ratio = 4
LR_shape = (120, 160, 3)
g_weight_name = "g_weight_01.h5"
d_weight_name = "d_weight_01.h5"

L_h, L_w, channels = LR_shape
H_h = L_h * ratio
H_w = L_w * ratio
# HR_shape = np.array([H_h, H_w, c])
HR_shape = (H_h, H_w, channels)

optimizer = Adam()

Using TensorFlow backend.
c:\users\robotics\anaconda3\envs\faster_rcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\robotics\anaconda3\envs\faster_rcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\robotics\anaconda3\envs\faster_rcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\robotics\an

In [2]:
def denormalize(img):
    img = (img + 1) * 127.5
    return img.astype(np.uint8)


def save_images(epoch, hr_img, lr_img, sr_img):
    hr_img = denormalize(hr_img)
    lr_img = denormalize(lr_img)
    sr_img = denormalize(sr_img)
    
    cv2.imwrite("../../images/output/" + str(epoch+1) + "_HRimage.png", hr_img)
    cv2.imwrite("../../images/output/" + str(epoch+1) + "_LRimage.png", lr_img)
    cv2.imwrite("../../images/output/" + str(epoch+1) + "_SRimage.png", sr_img)

    
def calc_psnr(img1: np.ndarray, img2: np.ndarray):
    def convert(img):
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    def extract_y(image: np.ndarray) -> np.ndarray:
        if image.ndim == 2:
            return image
        image = image.astype(np.int32)
        return ((image[:, :, 2] * 65.481 / 255.
                  + image[:, :, 1] * 128.553 / 255.
                  + image[:, :, 0] * 24.966 / 255.) + 16).astype(np.int32)


    def psnr(img1, img2):
        mse = np.mean((img1 - img2) ** 2)
        if mse == 0:
            return 100
        PIXEL_MAX = 255.0
        return 10 * math.log10(PIXEL_MAX * PIXEL_MAX / mse)

    img1_conv=convert(img1)
    img2_conv=convert(img2)

    # BGR -> YCrCb
    # 画像はcv2.imreadで読まれている前提 [0, 255]
    y1 = extract_y(img1_conv)
    y2 = extract_y(img2_conv)
    # 周囲のcropping
    # assert y1.shape == y2.shape
    h, w = y1.shape
    cr = ratio
    cropped_y1 = y1[cr:h - cr, cr:w - cr].astype(np.float64)
    cropped_y2 = y2[cr:h - cr, cr:w - cr].astype(np.float64)

    # psnr
    psnr_val = psnr(cropped_y1, cropped_y2)
    return psnr_val


def load_data(batch_size):

    files = glob.glob("../../images/train/*.png", recursive=True)
    batch_images = random.sample(files, batch_size)

    hr_imgs = []
    lr_imgs = []
    for img_path in batch_images:
        img = Image.open(img_path)

        hr_img = img.resize((H_w, H_h))  #(64, 64)
        lr_img = img.resize((L_w, L_h))
        hr_img = np.array(hr_img)
        #img_hr = (img_hr - 127.5) / 127.5
        lr_img = np.array(lr_img)
        #img_lr = (img_lr - 127.5) / 127.5

        hr_imgs.append(hr_img)
        lr_imgs.append(lr_img)

    hr_imgs = np.array(hr_imgs) / 127.5 - 1.
    lr_imgs = np.array(lr_imgs) / 127.5 - 1.

    return hr_imgs, lr_imgs
      

In [3]:

def pixel_shuffle(in_map, h, w, c):#((120, 160, 12), 120, 160, 3)
    
    x = Reshape((h, w, 2, 2, c))(in_map)
    x = Permute((3, 1, 4, 2, 5))(x)
    out_map = Reshape((2 * h, 2 * w, c))(x)
    
    return out_map


def upsampling(in_map, h, w, c):#((, 120, 160, 3), g:120, 160 , 64
    
    x = Conv2D(filters = 4 * c, 
                     kernel_size = 3,
                     strides = 1,
                     padding = "same")(in_map)# x: 120, 160, 12
    x = pixel_shuffle(x, h, w, c)# ( , 120, 160, 3)
    out_map = PReLU()(x)
    
    return out_map


def residual_block(in_map):
    x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(in_map)
    x = BatchNormalization()(x)
#     x = PReLU()(x)
    x = LeakyReLU(alpha = 0.2)(x)
    x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(x)
    x = BatchNormalization()(x)
    out_map = Add()([x, in_map])
    return out_map


def d_block(in_map, filters, kernel_size, strides, padding):
    d = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = padding)(in_map)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha = 0.2)(d)
    return d


def deconv2d(layer_input):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input) #
            u = Conv2D(120, kernel_size=3, strides=1, padding='same')(u) #
            u = Activation('relu')(u)
            return u


In [4]:
def build_generator():
    input_img = Input(shape = LR_shape)
    middle = Conv2D(filters = 64, kernel_size = 9, strides = 1, padding = "same")(input_img)
    middle = LeakyReLU(alpha = 0.2)(middle)
    
    g = residual_block(middle)
    for _ in range(4):
        g = residual_block(g)

    g = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(g)
    g = BatchNormalization()(g)
    g = Add()([g, middle])

    n = ratio
    i = 1
    while(n % 2 == 0):
        g = deconv2d(g)
#         g = upsampling(g, L_h * i, L_w * i, channels)#(g, 120, 160, 3), g:120, 160 , 64
#         i = i * 2
        n = n // 2

    output_img = Conv2D(filters = 3, kernel_size = 9, strides = 1, padding = "same")(g)

    return Model(input_img, output_img)


#------------------------------------------------------------------------#


def build_discriminator():
    input_img = Input(shape = HR_shape)
    
    d = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(input_img)
    d = d_block(d, filters = 64, kernel_size = 3, strides = 2, padding = "same")
    d = d_block(d, filters = 128, kernel_size = 3, strides = 1, padding = "same")
    d = d_block(d, filters = 128, kernel_size = 3, strides = 2, padding = "same")
    d = d_block(d, filters = 256, kernel_size = 3, strides = 1, padding = "same")
    d = d_block(d, filters = 256, kernel_size = 3, strides = 2, padding = "same")
    d = d_block(d, filters = 512, kernel_size = 3, strides = 1, padding = "same")
    d = d_block(d, filters = 512, kernel_size = 3, strides = 2, padding = "same")
    d = Flatten()(d)
    d = Dense(256)(d)
    d = LeakyReLU(alpha = 0.2)(d)
    output = Dense(1, activation = "sigmoid")(d)

    return Model(input_img, output)

#------------------------------------------------------------------------#

def build_vgg():
    vgg = VGG19(include_top = False)
    return Model(vgg.input, vgg.layers[9].output)
    
#------------------------------------------------------------------------#

def combined(generator, discriminator, vgg):
    input_img = Input(shape = LR_shape)
    fake_img = generator(input_img)
    
    validity = discriminator(fake_img)
    features = vgg(fake_img)
    
    return Model(input_img, [validity, features])




In [5]:
losses = []
epochs_checkpoint = []
psnr = []

def train(epochs, batch_size, interval):
    
    start_time = datetime.datetime.now()
    
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
        
        real_imgs, lr_imgs = load_data(batch_size)
        fake_imgs = generator.predict(lr_imgs)
        
        #Dの訓練
        d_loss_real = discriminator.train_on_batch(real_imgs, real)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        #Gの訓練
        vgg_features = vgg.predict(real_imgs)
        g_loss = srgan.train_on_batch(lr_imgs, [real, vgg_features])
        
        time = datetime.datetime.now() - start_time
        print("%d time: %s" % (epoch+1, time))
        
        if (epoch+1) % interval == 0:
            print("epoch: %d" % (epoch+1))
            losses.append((d_loss, g_loss))
            epochs_checkpoint.append(epoch+1)
            save_images(epoch, real_imgs[0], lr_imgs[0], fake_imgs[0])
#             psnr.append(calc_psnr(real_imgs[0], fake_imgs[0]))
            
    generator.save_weights(g_weight_name)
    discriminator.save_weights(d_weight_name)
    print("save seights")
        

In [6]:
def generator_train(epochs, batch_size, interval):
    start_time = datetime.datetime.now()
    generator.compile(loss = "mse",
                      optimizer = optimizer,
                      metrics = ["accuracy"])
    
    for epoch in range(epochs):
        
        real_imgs, lr_imgs = load_data(batch_size)
        fake_imgs = generator.predict(lr_imgs)
        
        #Gの訓練
        g_loss = generator.train_on_batch(lr_imgs, real_imgs)
        
        time = datetime.datetime.now() - start_time
        print("%d time: %s" % (epoch+1, time))
        
        if (epoch+1) % interval == 0:
            save_images(epoch, real_imgs[0], lr_imgs[0], fake_imgs[0])

    generator.save_weights("generator_first_weight.h5")
    print("save seights")

In [7]:
#------------------------------------
# メインプログラム
#------------------------------------

#Discriminator
discriminator = build_discriminator()
discriminator.compile(loss = "mse",
                      optimizer = optimizer,
                      metrics = ["accuracy"])
discriminator.summary()

#Generator どちらかを使う
#新規作成
generator = build_generator()
generator.summary()
#読み込み
# generator = load_model("generator_first_weight.h5")




Instructions for updating:
keep_dims is deprecated, use keepdims instead

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 480, 640, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 480, 640, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 240, 320, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 240, 320, 64)      256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 240, 320, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 240, 320, 128)     73856     
_________________________________________________________________

In [8]:
#generator_train(epochs = 10000, batch_size = 1, interval = 500)


In [9]:
vgg = build_vgg()
vgg.trainable = False
discriminator.trainable = False
srgan = combined(generator, discriminator, vgg)
srgan.compile(loss=['binary_crossentropy', 'mse'],
                              loss_weights=[1e-3, 1],
                              optimizer=optimizer)
srgan.summary()




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 120, 160, 3)   0                                            
____________________________________________________________________________________________________
model_2 (Model)                  (None, 480, 640, 3)   652763                                       
____________________________________________________________________________________________________
model_1 (Model)                  (None, 1)             161979713                                    
____________________________________________________________________________________________________
model_3 (Model)                  multiple              1735488                                 

In [10]:
train(epochs = 10000, batch_size = 1, interval = 500)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
1 time: 0:00:15.971169
2 time: 0:00:16.621798
3 time: 0:00:17.273485
4 time: 0:00:17.924252
5 time: 0:00:18.563409
6 time: 0:00:19.214089
7 time: 0:00:19.868849
8 time: 0:00:20.502345
9 time: 0:00:21.152935
10 time: 0:00:21.812534
11 time: 0:00:22.446765
12 time: 0:00:23.099865
13 time: 0:00:23.752604
14 time: 0:00:24.390671
15 time: 0:00:25.049269
16 time: 0:00:25.708867
17 time: 0:00:26.359141
18 time: 0:00:27.033050
19 time: 0:00:27.688350
20 time: 0:00:28.303129
21 time: 0:00:28.917899
22 time: 0:00:29.541040
23 time: 0:00:30.159681
24 time: 0:00:30.762232
25 time: 0:00:31.380576
26 time: 0:00:31.995605
27 time: 0:00:32.610584
28 time: 0:00:33.229600
29 time: 0:00:33.835574
30 time: 0:00:34.450537
31 time: 0:00:35.068374
32 time: 0:00:35.668568
33 time: 0:00:36.287418
34 time: 0:00:36.903191
35 time: 0:00:37.518329
36 time: 0:00:38.138672
37 time: 0:00:38.753823
38 

318 time: 0:03:33.871368
319 time: 0:03:34.497397
320 time: 0:03:35.115857
321 time: 0:03:35.731621
322 time: 0:03:36.348326
323 time: 0:03:36.959871
324 time: 0:03:37.573454
325 time: 0:03:38.182736
326 time: 0:03:38.810932
327 time: 0:03:39.426808
328 time: 0:03:40.033766
329 time: 0:03:40.661862
330 time: 0:03:41.272285
331 time: 0:03:41.883729
332 time: 0:03:42.504833
333 time: 0:03:43.127573
334 time: 0:03:43.734716
335 time: 0:03:44.361782
336 time: 0:03:44.972251
337 time: 0:03:45.593764
338 time: 0:03:46.214746
339 time: 0:03:46.825389
340 time: 0:03:47.434638
341 time: 0:03:48.065871
342 time: 0:03:48.676328
343 time: 0:03:49.297237
344 time: 0:03:49.917079
345 time: 0:03:50.536072
346 time: 0:03:51.165878
347 time: 0:03:51.793975
348 time: 0:03:52.421211
349 time: 0:03:53.069364
350 time: 0:03:53.718739
351 time: 0:03:54.340735
352 time: 0:03:54.966738
353 time: 0:03:55.576263
354 time: 0:03:56.210562
355 time: 0:03:56.855538
356 time: 0:03:57.498191
357 time: 0:03:58.142572


646 time: 0:07:06.266314
647 time: 0:07:06.923911
648 time: 0:07:07.586016
649 time: 0:07:08.221886
650 time: 0:07:08.877596
651 time: 0:07:09.536194
652 time: 0:07:10.167900
653 time: 0:07:10.834423
654 time: 0:07:11.490749
655 time: 0:07:12.120994
656 time: 0:07:12.788600
657 time: 0:07:13.443271
658 time: 0:07:14.074067
659 time: 0:07:14.738281
660 time: 0:07:15.394381
661 time: 0:07:16.027149
662 time: 0:07:16.688186
663 time: 0:07:17.342779
664 time: 0:07:17.996950
665 time: 0:07:18.643225
666 time: 0:07:19.302823
667 time: 0:07:19.947908
668 time: 0:07:20.593744
669 time: 0:07:21.252846
670 time: 0:07:21.906440
671 time: 0:07:22.543960
672 time: 0:07:23.202595
673 time: 0:07:23.858695
674 time: 0:07:24.493879
675 time: 0:07:25.153477
676 time: 0:07:25.810579
677 time: 0:07:26.448814
678 time: 0:07:27.107523
679 time: 0:07:27.767122
680 time: 0:07:28.406755
681 time: 0:07:29.066858
682 time: 0:07:29.724959
683 time: 0:07:30.363669
684 time: 0:07:31.023604
685 time: 0:07:31.684708


974 time: 0:10:40.837054
975 time: 0:10:41.502216
976 time: 0:10:42.148862
977 time: 0:10:42.807998
978 time: 0:10:43.470878
979 time: 0:10:44.118598
980 time: 0:10:44.781974
981 time: 0:10:45.441671
982 time: 0:10:46.087345
983 time: 0:10:46.753702
984 time: 0:10:47.419436
985 time: 0:10:48.071650
986 time: 0:10:48.725549
987 time: 0:10:49.391153
988 time: 0:10:50.040375
989 time: 0:10:50.702513
990 time: 0:10:51.369358
991 time: 0:10:52.009088
992 time: 0:10:52.673700
993 time: 0:10:53.336806
994 time: 0:10:53.977813
995 time: 0:10:54.642527
996 time: 0:10:55.306129
997 time: 0:10:55.962135
998 time: 0:10:56.615762
999 time: 0:10:57.300382
1000 time: 0:10:57.946481
epoch: 1000
1001 time: 0:10:58.612241
1002 time: 0:10:59.277594
1003 time: 0:10:59.931245
1004 time: 0:11:00.588500
1005 time: 0:11:01.270454
1006 time: 0:11:01.946067
1007 time: 0:11:02.581149
1008 time: 0:11:03.194545
1009 time: 0:11:03.811115
1010 time: 0:11:04.427788
1011 time: 0:11:05.044573
1012 time: 0:11:05.673418


1290 time: 0:13:56.897480
1291 time: 0:13:57.511082
1292 time: 0:13:58.124614
1293 time: 0:13:58.743324
1294 time: 0:13:59.358970
1295 time: 0:13:59.974523
1296 time: 0:14:00.587789
1297 time: 0:14:01.195255
1298 time: 0:14:01.809542
1299 time: 0:14:02.422807
1300 time: 0:14:03.051836
1301 time: 0:14:03.652804
1302 time: 0:14:04.281986
1303 time: 0:14:04.895296
1304 time: 0:14:05.512659
1305 time: 0:14:06.130624
1306 time: 0:14:06.744965
1307 time: 0:14:07.360443
1308 time: 0:14:07.974780
1309 time: 0:14:08.591350
1310 time: 0:14:09.204669
1311 time: 0:14:09.816977
1312 time: 0:14:10.430336
1313 time: 0:14:11.047703
1314 time: 0:14:11.661023
1315 time: 0:14:12.274358
1316 time: 0:14:12.895831
1317 time: 0:14:13.496818
1318 time: 0:14:14.126929
1319 time: 0:14:14.740255
1320 time: 0:14:15.353705
1321 time: 0:14:15.971454
1322 time: 0:14:16.585664
1323 time: 0:14:17.203231
1324 time: 0:14:17.816721
1325 time: 0:14:18.432290
1326 time: 0:14:19.046699
1327 time: 0:14:19.660437
1328 time: 0

...

7597 time: 1:19:06.965277
7598 time: 1:19:07.566476
7599 time: 1:19:08.195570
7600 time: 1:19:08.809110
7601 time: 1:19:09.406972
7602 time: 1:19:10.024391
7603 time: 1:19:10.637730
7604 time: 1:19:11.257020
7605 time: 1:19:11.873764
7606 time: 1:19:12.487122
7607 time: 1:19:13.101190
7608 time: 1:19:13.703034
7609 time: 1:19:14.320740
7610 time: 1:19:14.934382
7611 time: 1:19:15.550060
7612 time: 1:19:16.164770
7613 time: 1:19:16.781481
7614 time: 1:19:17.398276
7615 time: 1:19:18.012795
7616 time: 1:19:18.629459
7617 time: 1:19:19.248280
7618 time: 1:19:19.865352
7619 time: 1:19:20.478729
7620 time: 1:19:21.094459
7621 time: 1:19:21.696626
7622 time: 1:19:22.309986
7623 time: 1:19:22.928191
7624 time: 1:19:23.542552
7625 time: 1:19:24.159624
7626 time: 1:19:24.773160
7627 time: 1:19:25.388763
7628 time: 1:19:26.003288
7629 time: 1:19:26.620066
7630 time: 1:19:27.233468
7631 time: 1:19:27.846918
7632 time: 1:19:28.460364
7633 time: 1:19:29.077927
7634 time: 1:19:29.681014
7635 time: 1

7913 time: 1:22:21.087294
7914 time: 1:22:21.700898
7915 time: 1:22:22.318443
7916 time: 1:22:22.931821
7917 time: 1:22:23.549536
7918 time: 1:22:24.163983
7919 time: 1:22:24.777366
7920 time: 1:22:25.394127
7921 time: 1:22:26.007550
7922 time: 1:22:26.620927
7923 time: 1:22:27.234457
7924 time: 1:22:27.852038
7925 time: 1:22:28.465504
7926 time: 1:22:29.078861
7927 time: 1:22:29.692273
7928 time: 1:22:30.305728
7929 time: 1:22:30.919079
7930 time: 1:22:31.540376
7931 time: 1:22:32.157369
7932 time: 1:22:32.770833
7933 time: 1:22:33.387501
7934 time: 1:22:34.000941
7935 time: 1:22:34.614294
7936 time: 1:22:35.227704
7937 time: 1:22:35.844478
7938 time: 1:22:36.457920
7939 time: 1:22:37.071642
7940 time: 1:22:37.688654
7941 time: 1:22:38.302157
7942 time: 1:22:38.915620
7943 time: 1:22:39.533212
7944 time: 1:22:40.150778
7945 time: 1:22:40.752999
7946 time: 1:22:41.366414
7947 time: 1:22:41.979802
7948 time: 1:22:42.597296
7949 time: 1:22:43.216092
7950 time: 1:22:43.832902
7951 time: 1

8228 time: 1:25:34.720261
8229 time: 1:25:35.337852
8230 time: 1:25:35.951129
8231 time: 1:25:36.564491
8232 time: 1:25:37.177791
8233 time: 1:25:37.796730
8234 time: 1:25:38.397979
8235 time: 1:25:39.011494
8236 time: 1:25:39.640534
8237 time: 1:25:40.241520
8238 time: 1:25:40.854922
8239 time: 1:25:41.468337
8240 time: 1:25:42.097369
8241 time: 1:25:42.699715
8242 time: 1:25:43.321243
8243 time: 1:25:43.935879
8244 time: 1:25:44.553239
8245 time: 1:25:45.166664
8246 time: 1:25:45.782000
8247 time: 1:25:46.395022
8248 time: 1:25:47.001147
8249 time: 1:25:47.614579
8250 time: 1:25:48.230293
8251 time: 1:25:48.844793
8252 time: 1:25:49.442682
8253 time: 1:25:50.071761
8254 time: 1:25:50.673731
8255 time: 1:25:51.287143
8256 time: 1:25:51.900597
8257 time: 1:25:52.514016
8258 time: 1:25:53.132875
8259 time: 1:25:53.747295
8260 time: 1:25:54.360727
8261 time: 1:25:54.974186
8262 time: 1:25:55.591685
8263 time: 1:25:56.205250
8264 time: 1:25:56.808385
8265 time: 1:25:57.421819
8266 time: 1

8543 time: 1:28:51.879503
8544 time: 1:28:52.517220
8545 time: 1:28:53.174907
8546 time: 1:28:53.834104
8547 time: 1:28:54.470181
8548 time: 1:28:55.127281
8549 time: 1:28:55.799891
8550 time: 1:28:56.453658
8551 time: 1:28:57.111346
8552 time: 1:28:57.782493
8553 time: 1:28:58.425073
8554 time: 1:28:59.081292
8555 time: 1:28:59.740891
8556 time: 1:29:00.396982
8557 time: 1:29:01.059821
8558 time: 1:29:01.724931
8559 time: 1:29:02.361932
8560 time: 1:29:03.023533
8561 time: 1:29:03.682635
8562 time: 1:29:04.323891
8563 time: 1:29:04.982488


KeyboardInterrupt: 

In [9]:
for epoch in range(len(epochs_checkpoint)):
    print("epoch: %d  d_loss: %.3f  g_loss: %.3f  psnr: %.3f"
         .format(epochs_checkpoint[epoch],
                losses[epoch, 0],
                losses[epoch, 1],
                psnr[epoch]))
    

In [11]:
print(len(losses))

0


In [20]:
from PIL import Image
D = Discriminator(480, 640, 3)
img = Image.open("../../images/train/image_1.png")
img = np.array(img)
valid = D.discriminate(img)
print(valid)

Tensor("dense_2/Sigmoid:0", shape=(?, 30, 40, 1), dtype=float32)


In [ ]:
import tensorflow as tf

def depth_to_space():
    x = tf.random.normal((16, 64, 64, 12))
    y = tf.nn.depth_to_space(x, 2)
    print(y.shape) # (16, 128, 128, 3)
    
depth_to_space()

In [11]:
generator.save_weights(g_weight_name)
discriminator.save_weights(d_weight_name)